Custom type checker test - a new validator added to the existing type

In [1]:
from functools import partial

from schematics.exceptions import DataError, ValidationError
from schematics.models import Model
from schematics.types import *

Validators can be added to the existing type as functions.

In [2]:
def is_even(constraint, value, /):
    if constraint is True and value & 1 == 1:
        raise ValidationError('Int value must be even')
    if constraint is False and value & 1 == 0:
        raise ValidationError("Int value must not be even")

In [3]:
class Schema(Model):
    number = IntType(validators=[partial(is_even, True)])

In [4]:
instance = Schema({"number": 4})
instance.validate()

In [5]:
instance = Schema({})
instance.validate()

In [6]:
instance = Schema({"number": 3})
try:
    instance.validate()
except DataError as e:
    print(e.errors)

{'number': ValidationError([ErrorMessage('Int value must be even', None)])}


Validators can also be added as methods in the subclassed type.

In [7]:
class CustomIntType(IntType):
    def __init__(self, *, is_even=None, **kwargs):
        super().__init__(**kwargs)
        self.is_even = is_even

    def validate_is_even(self, value):
        if self.is_even is True and value & 1 == 1:
            raise ValidationError("Int value must be even")
        if self.is_even is False and value & 1 == 0:
            raise ValidationError("Int value must not be even")

In [8]:
class Schema(Model):
    number = CustomIntType(is_even=True)

In [9]:
instance = Schema({"number": 4})
instance.validate()

In [10]:
instance = Schema({"number": 3})
try:
    instance.validate()
except DataError as e:
    print(e.errors)

{'number': ValidationError([ErrorMessage('Int value must be even', None)])}
